In [ ]:
# LoRA Fine-Tuning Script for phi-3-mini-4k-instruct using PEFT
# Assumes you have combined_phi3_wealth_dataset.jsonl

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
import torch
import json

# Load tokenizer and model (Phi-3-mini)
model_name = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)

# Prepare for 4-bit quantization + LoRA
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Load combined JSONL dataset
def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f if line.strip()]

data = load_jsonl("phi3_wealth_data/combined_phi3_wealth_dataset.jsonl")
dataset = Dataset.from_list(data)

# Tokenize
def tokenize(example):
    prompt = example["prompt"]
    response = example["response"]
    text = f"<|user|>\n{prompt}\n<|assistant|>\n{response}"
    return tokenizer(text, truncation=True, padding="max_length", max_length=2048)

tok_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

# Training config
training_args = TrainingArguments(
    output_dir="phi3-wealth-lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    train_dataset=tok_dataset,
    args=training_args,
    data_collator=data_collator,
)

trainer.train()

# Save the LoRA adapter
model.save_pretrained("phi3-wealth-lora")
tokenizer.save_pretrained("phi3-wealth-lora")
